In [1]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.nr_scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
startURL = 'https://newrepublic.com/'
domain = 'newrepublic.com'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [5]:
len(urls)

1

In [6]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 10:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html, 
                }

            # insert dict into db document
            try:
                htmlCollection.insert_one(data)
            except Exception as e:
                print(f'htmlCollection Insert: {e}')
                done_urls.add(url)
                continue
            
            # record url in "done" collection
            done_urls.add(url)
            print(f'done_urls: {len(done_urls)}')
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            done_urls.add(url)
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  https://newrepublic.com/
Elapsed time: 6.706806898117065
done_urls: 1
Extracted links: 58

Number of Urls: 1
Process Time: 7.143128871917725
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  53
LOOP counter:  1
--------------------
URL in for loop:  https://newrepublic.com/article/149145/jesus-lock-up
Elapsed time: 5.753616094589233
done_urls: 2
Extracted links: 80
URL in for loop:  https://newrepublic.com/article/149185/colombia-keeps-electing-presidents-tied-murderers
Elapsed time: 5.826749563217163
done_urls: 3
Extracted links: 112
URL in for loop:  https://newrepublic.com/pages/privacy
Elapsed time: 5.629953861236572
done_urls: 4
Extracted links: 116
URL in for loop:  https://newrepublic.com/minutes
Elapsed time: 6.416240930557251
done_urls: 5
Extracted links: 199
URL in for loop:  https://newrepublic.com/authors/steven-cohen
Elapsed time: 6.938741683959961
done_urls: 6
Extracted links: 219
URL in for loop:  https://

ScrapeHTML: No connection adapters were found for 'mailto:?subject=&body=Witnesses%20for%0Athe%20Future%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%0Ahttps%3A%2F%2Fnewrepublic.com%2Farticle%2F148904%2Fslave-trade-last-survivor-zora-neale-hurston'
Elapsed time: 5.006103038787842
Main Loop: HTML not extracted
URL in for loop:  mailto:gvyse@newrepublic.com
ScrapeHTML: No connection adapters were found for 'mailto:gvyse@newrepublic.com'
Elapsed time: 5.004669427871704
Main Loop: HTML not extracted
URL in for loop:  https://newrepublic.com/article/148473/jordan-petersons-tired-old-myths
Elapsed time: 5.8155293464660645
done_urls: 58
Extracted links: 1163
URL in for loop:  https://newrepublic.com/tags/children
Elapsed time: 6.680374383926392
done_urls: 59
Extracted links: 1198
URL in for loop:  https://newrepublic.com/authors/ioan-grillo
Elapsed time: 5.789398670196533
done_urls: 60
Extracted links: 1200
URL in for loop:  mailto:?subject=The%20New%20Republic%20Minute

ScrapeHTML: No connection adapters were found for 'mailto:?subject=&body=The%20Consequences%20of%20the%20Supreme%20Court%E2%80%99s%20Punt%20on%20Gerrymandering%0A%0Ahttps%3A%2F%2Fnewrepublic.com%2Farticle%2F149158%2Fconsequences-supreme-courts-punt-gerrymandering'
Elapsed time: 5.001517295837402
Main Loop: HTML not extracted
URL in for loop:  https://newrepublic.com/tags/review
Elapsed time: 7.534014463424683
done_urls: 101
Extracted links: 1830
URL in for loop:  https://newrepublic.com/article/123590/sex-and-the-muslim-feminist
Elapsed time: 5.8759918212890625
done_urls: 102
Extracted links: 1851
URL in for loop:  https://newrepublic.com/article/148399/scourge-trumpism-conservative-journalism
Elapsed time: 5.7912585735321045
done_urls: 103
Extracted links: 1872
URL in for loop:  https://newrepublic.com/article/131840/bernies-foreign-policy-important-ever
Elapsed time: 5.834233522415161
done_urls: 104
Extracted links: 1901
URL in for loop:  mailto:?subject=The%20New%20Republic%20Minute

Elapsed time: 5.591148376464844
done_urls: 142
Extracted links: 2241
URL in for loop:  https://newrepublic.com/tags/religion
Elapsed time: 6.795010328292847
done_urls: 143
Extracted links: 2257
URL in for loop:  https://newrepublic.com/tags/funeral
Elapsed time: 5.841176986694336
done_urls: 144
Extracted links: 2260
URL in for loop:  https://newrepublic.com/article/148448/new-republic-june-issue-new-battleground
Elapsed time: 5.791760444641113
done_urls: 145
Extracted links: 2262
URL in for loop:  https://newrepublic.com/article/99316/arthur-conan-doyle-sherlock-holmes-michael-dirda-christopher-sandford-houdini
Elapsed time: 5.843653678894043
done_urls: 146
Extracted links: 2267
URL in for loop:  https://newrepublic.com/authors/josephine-livingstone?page=2
Elapsed time: 6.842516660690308
done_urls: 147
Extracted links: 2281
URL in for loop:  https://newrepublic.com/tags/wisconsin
Elapsed time: 7.2425596714019775
done_urls: 148
Extracted links: 2307
URL in for loop:  https://newrepublic

Elapsed time: 7.4914045333862305
done_urls: 189
Extracted links: 2883
URL in for loop:  https://newrepublic.com/tags/books-and-arts
Elapsed time: 6.868468284606934
done_urls: 190
Extracted links: 2913
URL in for loop:  mailto:swhalen@newrepublic.com
ScrapeHTML: No connection adapters were found for 'mailto:swhalen@newrepublic.com'
Elapsed time: 5.006958246231079
Main Loop: HTML not extracted
URL in for loop:  https://newrepublic.com/tags/july-august-2018
Elapsed time: 5.8415539264678955
done_urls: 192
Extracted links: 2914
URL in for loop:  https://newrepublic.com/article/141698/africas-last-colony-western-sahara-photo-essay
Elapsed time: 5.802353620529175
done_urls: 193
Extracted links: 2935
URL in for loop:  https://newrepublic.com/authors/gabriel-winant
Elapsed time: 5.770190954208374
done_urls: 194
Extracted links: 2935
URL in for loop:  https://newrepublic.com/tags/climate-change?page=2
Elapsed time: 7.028367757797241
done_urls: 195
Extracted links: 2948
URL in for loop:  https://

...

Elapsed time: 7.026339530944824
done_urls: 317
Extracted links: 4081
URL in for loop:  mailto:tross@newrepublic.com
ScrapeHTML: No connection adapters were found for 'mailto:tross@newrepublic.com'
Elapsed time: 5.002445936203003
Main Loop: HTML not extracted
URL in for loop:  https://newrepublic.com/authors/anthony-jean
Elapsed time: 5.999478101730347
done_urls: 319
Extracted links: 4081
URL in for loop:  https://newrepublic.com/article/148190/know-healing-crystals-come-from
Elapsed time: 5.660468101501465
done_urls: 320
Extracted links: 4132
URL in for loop:  https://newrepublic.com/tags/transit
Elapsed time: 6.030020713806152
done_urls: 321
Extracted links: 4136
URL in for loop:  https://newrepublic.com/article/143508/crossing-record-number-refugees-braving-deadly-voyage-libya-italy
Elapsed time: 6.08600926399231
done_urls: 322
Extracted links: 4143
URL in for loop:  https://newrepublic.com/article/148879/immigration-forces-beyond-trumps-control
Elapsed time: 6.034072399139404
done_u

Elapsed time: 6.038995742797852
done_urls: 369
Extracted links: 4568
URL in for loop:  https://newrepublic.com/authors/robert-capa
Elapsed time: 5.752797842025757
done_urls: 370
Extracted links: 4568
URL in for loop:  https://newrepublic.com/article/148308/centrist-grievance-victim-politics
Elapsed time: 5.8264501094818115
done_urls: 371
Extracted links: 4584
URL in for loop:  https://newrepublic.com/tags/dating
Elapsed time: 6.674486398696899
done_urls: 372
Extracted links: 4606
URL in for loop:  https://newrepublic.com/authors/kim-kelly
Elapsed time: 5.781585931777954
done_urls: 373
Extracted links: 4607
URL in for loop:  https://www.linkedin.com/shareArticle?mini=true&url=https://newrepublic.com/article/149158/consequences-supreme-courts-punt-gerrymandering&title=The Consequences of the Supreme Court’s Punt on Gerrymandering&summary=The justices on Monday ensured that warped legislative maps will continue to erode American self-government. &utm_source=social&utm_medium=linkedin&utm_

Elapsed time: 6.080895900726318
done_urls: 411
Extracted links: 4880
URL in for loop:  https://newrepublic.com/tags/swimsuit
Elapsed time: 5.744057655334473
done_urls: 412
Extracted links: 4881
URL in for loop:  https://newrepublic.com/tags/media
Elapsed time: 6.724132537841797
done_urls: 413
Extracted links: 4888
URL in for loop:  https://newrepublic.com/article/143172/onboard-ice-air-immigration-enforcement-apparatus-obama-built-trump-exploits
Elapsed time: 5.779690504074097
done_urls: 414
Extracted links: 4898
URL in for loop:  https://newrepublic.com/authors/steven-cohen?page=2
Elapsed time: 6.6863038539886475
done_urls: 415
Extracted links: 4915
URL in for loop:  https://newrepublic.com/article/118615/richard-linklaters-boyhood-stunning-experiment
Elapsed time: 5.971248388290405
done_urls: 416
Extracted links: 4927
URL in for loop:  https://newrepublic.com/tags/economics
Elapsed time: 7.089405536651611
done_urls: 417
Extracted links: 4940
URL in for loop:  https://newrepublic.com/

Elapsed time: 5.793174982070923
done_urls: 456
Extracted links: 5296
URL in for loop:  https://newrepublic.com/tags/asylum
Elapsed time: 6.292691946029663
done_urls: 457
Extracted links: 5307
URL in for loop:  https://newrepublic.com/article/148304/rough-justice-america-over-policed
Elapsed time: 5.890636444091797
done_urls: 458
Extracted links: 5315
URL in for loop:  https://newrepublic.com/article/148413/dawn-new-era-nuclear-proliferation
Elapsed time: 5.76619029045105
done_urls: 459
Extracted links: 5332
URL in for loop:  https://newrepublic.com/article/148616/cruelty-trumps-immigration-policy
Elapsed time: 5.902231931686401
done_urls: 460
Extracted links: 5357
URL in for loop:  https://newrepublic.com/tags/true-crime
Elapsed time: 5.9753546714782715
done_urls: 461
Extracted links: 5360
URL in for loop:  https://newrepublic.com/authors/lisa-palmer
Elapsed time: 5.799972772598267
done_urls: 462
Extracted links: 5360
URL in for loop:  https://newrepublic.com/article/148931/you-not-wan

Elapsed time: 5.765606164932251
done_urls: 502
Extracted links: 5746
URL in for loop:  https://newrepublic.com/pages/faq#question-6
Elapsed time: 5.613095283508301
done_urls: 503
Extracted links: 5746
URL in for loop:  mailto:film@newrepublic.com
ScrapeHTML: No connection adapters were found for 'mailto:film@newrepublic.com'
Elapsed time: 5.003207445144653
Main Loop: HTML not extracted

Number of Urls: 450
Process Time: 2704.0410001277924
--------------------------------------------------------------
done_urls:  504
(urls - done_urls):  0
LOOP counter:  3
--------------------
